## Tacotron 2 inference code 
Edit the variables **checkpoint_path** and **text** to match yours and run the entire code to generate plots of mel outputs, alignments and audio synthesis from the generated mel-spectrogram using Griffin-Lim.

#### Import libraries and setup matplotlib

In [1]:
import matplotlib
import matplotlib.pylab as plt
import IPython.display as ipd
import sys
sys.path.append('waveglow/') # 终于知道这个的作用了，因为下方的Denoiser是来自这的
import numpy as np
import torch
from hparams import create_hparams
from model import Tacotron2
from layers import TacotronSTFT, STFT
from audio_processing import griffin_lim
from train import load_model
from text import text_to_sequence
from denoiser import Denoiser # 此处警告在实际运行时会消除
from matplotlib.pyplot import savefig # wsy add

In [2]:
def plot_data(data, figsize=(16, 4)):
    fig, axes = plt.subplots(1, len(data), figsize=figsize)
    for i in range(len(data)):
        # axes[i].imshow(data[i], aspect='auto', origin='bottom', 
        #                interpolation='none') # 报错：ValueError: 'bottom' is not a valid value for origin; supported values are 'upper', 'lower'
        axes[i].imshow(data[i], aspect='auto', origin='lower', 
                       interpolation='none')  # wsy fix
    savefig("1.jpg") # wsy add 

#### Setup hparams

In [3]:
hparams = create_hparams()
hparams.sampling_rate = 22050

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



#### Load model from checkpoint

In [4]:
checkpoint_path = "tacotron2_statedict.pt"
model = load_model(hparams)
model.load_state_dict(torch.load(checkpoint_path)['state_dict'])
# _ = model.cuda().eval().half()
_ = model.cuda().eval()

#### Load WaveGlow for mel2audio synthesis and denoiser

In [5]:
waveglow_path = 'waveglow_256channels_universal_v5.pt' # wsy fix
waveglow = torch.load(waveglow_path)['model']
# waveglow.cuda().eval().half()
waveglow.cuda().eval()
for k in waveglow.convinv:
    k.float()
denoiser = Denoiser(waveglow)

/root/anaconda3/envs/exp6/lib/python3.7/site-packages/torch/serialization.py:868: SourceChangeWarning: source code of class 'glow.WaveGlow' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/root/anaconda3/envs/exp6/lib/python3.7/site-packages/torch/serialization.py:868: SourceChangeWarning: source code of class 'torch.nn.modules.conv.ConvTranspose1d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/root/anaconda3/envs/exp6/lib/python3.7/site-packages/torch/serialization.py:868: SourceChangeWarning: source code of class 'torch.nn.modules.container.ModuleList' has changed. you can retrieve the original source code by accessing th

#### Prepare text input

In [6]:
text = "Waveglow is really awesome!"
# text="I'm wang shi yao"
sequence = np.array(text_to_sequence(text, ['english_cleaners']))[None, :]
sequence = torch.autograd.Variable(
    torch.from_numpy(sequence)).cuda().long()

#### Decode text input and plot results

In [7]:
mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)
plot_data((mel_outputs.float().data.cpu().numpy()[0],
           mel_outputs_postnet.float().data.cpu().numpy()[0],
           alignments.float().data.cpu().numpy()[0].T))

#### Synthesize audio from spectrogram using WaveGlow

In [8]:
with torch.no_grad():
    audio = waveglow.infer(mel_outputs_postnet, sigma=0.666)
print("audio[0]：",audio[0].data.cpu().numpy())
print("audic：",audio.data.cpu().numpy())
ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate) # jupter中用于播放音频的
# ipd.Audio(audio.data.cpu().numpy(), rate=hparams.sampling_rate) # jupter中用于播放音频的

audio[0]： [4.3893830e-05 2.1764739e-05 3.5195681e-05 ... 7.2815492e-05 4.0137948e-05
 8.1991951e-05]
audic： [[4.3893830e-05 2.1764739e-05 3.5195681e-05 ... 7.2815492e-05
  4.0137948e-05 8.1991951e-05]]


#### (Optional) Remove WaveGlow bias

In [10]:
audio_denoised = denoiser(audio, strength=0.01)[:, 0]
print("audio_denoised：",audio_denoised.cpu().numpy())
ipd.Audio(audio_denoised.cpu().numpy(), rate=hparams.sampling_rate) 

audio_denoised： [[-1.8478453e-05 -9.4773395e-06 -1.6318003e-05 ...  3.7058941e-05
   3.5279492e-05  3.6061010e-05]]
